<a href="https://colab.research.google.com/github/Cami5543/Trabajo-Finanzas-III/blob/main/TrabajoFinalFinanzasiii_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
%pip install fpdf
%pip install networkx
# =========================================================================
# === PASO 1: Importaciones, Configuración de Estilo y Descarga de Logo ===
# =========================================================================
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests
from fpdf import FPDF
import networkx as nx
import sys
from datetime import datetime

# Descargar imagen desde URL (logo Usach)
url_logo = "https://registro.usach.cl/imagen/UsachP2.png"
logo_path = "UsachP2.png"  # Directorio actual

if not os.path.exists(logo_path):
    r = requests.get(url_logo)
    if r.status_code == 200:
        with open(logo_path, "wb") as f:
            f.write(r.content)

# Aplicar un estilo profesional a todos los gráficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.ioff()

# === Función para obtener fecha en español ===
def obtener_fecha_en_espanol():
    now = datetime.now()
    meses = (
        "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
        "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
    )
    nombre_mes = meses[now.month - 1]
    return f"{now.day} de {nombre_mes} de {now.year}"

# =========================================================================
# === CÁLCULOS Y DATOS PARA CADA ACTIVO ===================================
# =========================================================================

# --- 1. Bonos ---
principal_bono = 1000
coupon_rate_bono = 0.05
years_to_maturity_bono = 5
payments_per_year_bono = 2
yield_to_maturity_bono = 0.06

def calculate_bond_price(principal, coupon_rate, years_to_maturity, payments_per_year, ytm):
    coupon_payment = (principal * coupon_rate) / payments_per_year
    total_payments = years_to_maturity * payments_per_year
    period_yield = ytm / payments_per_year
    pv_coupons = sum([coupon_payment / ((1 + period_yield)**i) for i in range(1, total_payments + 1)])
    pv_principal = principal / ((1 + period_yield)**total_payments)
    return pv_coupons + pv_principal

valor_bono = calculate_bond_price(principal_bono, coupon_rate_bono, years_to_maturity_bono, payments_per_year_bono, yield_to_maturity_bono)

resumen_bono = {
    "descripcion": "Un bono es un instrumento de deuda que promete pagos de intereses (cupones) periódicos y el reembolso del principal al vencimiento. Su valor presente se calcula descontando estos flujos de efectivo a la tasa de rendimiento al vencimiento (YTM).",
    "detalles": [
        f"Valor Nominal: ${principal_bono:,.2f}",
        f"Tasa de Cupón Anual: {coupon_rate_bono*100:.2f}%",
        f"Años al Vencimiento: {years_to_maturity_bono}",
        f"Pagos por Año: {payments_per_year_bono} (Semestral)",
        f"Rendimiento al Vencimiento (YTM): {yield_to_maturity_bono*100:.2f}%"
    ],
    "resultado": f"El Valor Presente del Bono es: ${valor_bono:,.2f}",
    "analisis": f"Dado que la YTM ({yield_to_maturity_bono*100:.2f}%) es mayor que la tasa de cupón ({coupon_rate_bono*100:.2f}%), el bono se valora por debajo de su valor nominal (con descuento), lo que indica que el precio de mercado es menor que su valor nominal para ofrecer el rendimiento deseado."
}

ytm_range_bono = np.linspace(0.01, 0.10, 100)
bond_prices_bono = [calculate_bond_price(principal_bono, coupon_rate_bono, years_to_maturity_bono, payments_per_year_bono, ytm) for ytm in ytm_range_bono]
plt.figure(figsize=(10, 6))
plt.plot(ytm_range_bono * 100, bond_prices_bono, label='Precio del Bono', color='darkgreen')
plt.axvline(yield_to_maturity_bono * 100, color='r', linestyle='--', label=f'YTM Actual: {yield_to_maturity_bono*100:.2f}%')
plt.axhline(valor_bono, color='blue', linestyle=':', label=f'Precio Actual: ${valor_bono:,.2f}')
plt.title('Bonos: Precio del Bono vs. Rendimiento al Vencimiento (YTM)')
plt.xlabel('Rendimiento al Vencimiento (YTM) (%)')
plt.ylabel('Precio del Bono ($)')
plt.legend()
plt.scatter(yield_to_maturity_bono * 100, valor_bono, color='red', s=100, zorder=5)
plt.annotate(f'({yield_to_maturity_bono*100:.2f}%, ${valor_bono:,.2f})', (yield_to_maturity_bono * 100, valor_bono), textcoords="offset points", xytext=(0,10), ha='center', color='red')
ruta_bonos = "bono_sensibilidad.png"
plt.savefig(ruta_bonos, bbox_inches='tight', dpi=150)
plt.close()

# --- 2. Acciones ---
dividend_actual_accion = 2.0
growth_rate_accion = 0.05
required_return_accion = 0.12

def calculate_stock_price_gordon(D0, g, r):
    if r <= g: return float('inf')
    return (D0 * (1 + g)) / (r - g)

valor_accion = calculate_stock_price_gordon(dividend_actual_accion, growth_rate_accion, required_return_accion)

resumen_accion = {
    "descripcion": "El Modelo de Crecimiento de Gordon (o Modelo de Descuento de Dividendos con Crecimiento Constante) valora una acción asumiendo que los dividendos crecerán a una tasa constante indefinidamente. Es una herramienta útil para estimar el valor intrínseco de empresas maduras con flujos de efectivo predecibles.",
    "detalles": [
        f"Dividendo Actual (D0): ${dividend_actual_accion:,.2f} por acción",
        f"Tasa de Crecimiento de Dividendos (g): {growth_rate_accion*100:.2f}%",
        f"Tasa de Rendimiento Requerida (r): {required_return_accion*100:.2f}%"
    ],
    "resultado": f"El Valor Intrínseco de la Acción es: ${valor_accion:,.2f}",
    "analisis": f"Este valor teórico de ${valor_accion:,.2f} es sensible a los cambios en la tasa de crecimiento esperada y a la tasa de rendimiento requerida por el mercado. Si el precio de mercado actual es significativamente diferente, podría indicar una oportunidad de inversión (si es menor) o una sobrevaloración (si es mayor). Es fundamental que r > g para que el modelo sea válido."
}

growth_rate_range_accion = np.linspace(0.01, required_return_accion - 0.01, 100)
stock_prices_accion = [calculate_stock_price_gordon(dividend_actual_accion, g, required_return_accion) for g in growth_rate_range_accion]
plt.figure(figsize=(10, 6))
plt.plot(growth_rate_range_accion * 100, stock_prices_accion, label='Precio de la Acción', color='darkblue')
plt.axvline(growth_rate_accion * 100, color='r', linestyle='--', label=f'Crecimiento Actual: {growth_rate_accion*100:.2f}%')
plt.axhline(valor_accion, color='green', linestyle=':', label=f'Precio Actual: ${valor_accion:,.2f}')
plt.title('Acciones: Precio de la Acción vs. Tasa de Crecimiento de Dividendos (g)')
plt.xlabel('Tasa de Crecimiento (g) (%)')
plt.ylabel('Precio de la Acción ($)')
plt.legend()
plt.scatter(growth_rate_accion * 100, valor_accion, color='red', s=100, zorder=5)
plt.annotate(f'({growth_rate_accion*100:.2f}%, ${valor_accion:,.2f})', (growth_rate_accion * 100, valor_accion), textcoords="offset points", xytext=(0,10), ha='center', color='red')
ruta_acciones = "accion_sensibilidad.png"
plt.savefig(ruta_acciones, bbox_inches='tight', dpi=150)
plt.close()

# --- 3. Unidad de Fomento (UF) ---
from datetime import date, timedelta
import matplotlib.pyplot as plt
import numpy as np

# Aplicar un estilo profesional a todos los gráficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.ioff() # Desactivar la visualización interactiva de gráficos

def calcular_uf_lineal(uf_base, ipc_mensual, dias_transcurridos, dias_totales_mes):
    """
    Calcula el valor de la UF de forma lineal para un período específico.
    Args:
        uf_base (float): Valor de la UF al inicio del período.
        ipc_mensual (float): IPC mensual (como decimal).
        dias_transcurridos (int): Número de días transcurridos desde el inicio del reajuste.
        dias_totales_mes (int): Número total de días en el mes al que corresponde el IPC.
    Returns:
        float: Valor estimado de la UF.
    """
    if dias_totales_mes == 0:
        return uf_base # Evitar división por cero
    return uf_base * (1 + (ipc_mensual * dias_transcurridos / dias_totales_mes))

# --- Datos para el cálculo de la UF (solo el IPC de mayo) ---

# Valor de la UF al 9 de junio de 2025 (valor base proporcionado)
uf_9_junio = 39212.19

# IPC de mayo de 2025. Este IPC es el que rige la UF desde el 9 de junio hasta el 8 de julio.
ipc_mayo = 0.002 # 0.2%

# Días en el mes del IPC de mayo (Mayo tiene 31 días)
# Sin embargo, el reajuste para el período 9-8 se calcula sobre 30 días normalmente.
# Para el reajuste diario de la UF, se usa el IPC mensual dividido por los días del mes del IPC.
# Pero si estamos proyectando de 9 a 8 y el IPC de mayo es el único que afecta el periodo de 30 días,
# entonces se aplica sobre el periodo completo.

# --- Cálculo de la UF al 8 de julio de 2025 ---

# El periodo del 9 de junio al 8 de julio es de 30 días.
# Durante este período, la UF se reajusta por el IPC de mayo (0.002).
dias_transcurridos_9_jun_a_8_jul = 30 # Del 9 de junio al 8 de julio hay 30 días.

# El IPC de mayo se aplica sobre 31 días (que son los días del mes de mayo).
dias_totales_mes_ipc_mayo = 31

uf_8_julio_calculado = calcular_uf_lineal(uf_9_junio, ipc_mayo, dias_transcurridos_9_jun_a_8_jul, dias_totales_mes_ipc_mayo)

# --- Preparar datos para la tabla y el gráfico ---
tabla_uf_data = [
    ["Fecha", "UF (CLP)"],
    ["9-Jun-2025", f"${uf_9_junio:,.2f}"],
    ["8-Jul-2025", f"${uf_8_julio_calculado:,.2f}"]
]

resumen_uf = {
    "descripcion": "La Unidad de Fomento (UF) es una unidad de cuenta reajustable según la inflación en Chile. Su valor se actualiza diariamente y refleja las variaciones del Índice de Precios al Consumidor (IPC).\n\nEste análisis presenta el valor de la UF para el 9 de junio de 2025 y una proyección para el 9 de julio de 2025, utilizando exclusivamente el IPC de mayo, ya que este rige el período completo de reajuste entre estas fechas (del 9 al 9).",
    "detalles": [
        f"UF al 9 de Junio de 2025 (Base): ${uf_9_junio:,.2f}",
        f"IPC Mayo 2025 (Aplicado): {ipc_mayo*100:.2f}%"
    ],
    "tabla": tabla_uf_data,
    "analisis": "La proyección lineal de la UF del 9 de junio al 8 de julio de 2025 se basa en el IPC de mayo. Esto refleja el ajuste inflacionario sobre el período de un mes exacto (30 días), demostrando cómo el valor de la UF se incrementa con la inflación esperada del mes anterior."
}

# --- Generar el gráfico de la UF ---
fechas_uf_labels = [row[0] for row in tabla_uf_data[1:]]
ufs_valores = [uf_9_junio, uf_8_julio_calculado]
plt.figure(figsize=(8, 5)) # Ajusta el tamaño para dos puntos
plt.plot(fechas_uf_labels, ufs_valores, marker='o', linestyle='-', color='darkgreen')
plt.title("Proyección UF: 9 de Junio a 8 de Julio de 2025 (Basado en IPC de Mayo)", fontsize=14)
plt.ylabel("UF (CLP)", fontsize=12)
plt.xticks(rotation=0, ha='center') # No es necesario rotar para solo dos etiquetas
plt.ylim(min(ufs_valores)*0.99, max(ufs_valores)*1.01)

# Añadir valores en cada punto del gráfico
for i, valor in enumerate(ufs_valores):
    plt.text(i, valor + 50, f"${valor:,.2f}", ha='center', fontsize=10, color='darkgreen', bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', boxstyle='round,pad=0.2'))
plt.tight_layout()
ruta_uf = "uf_proyeccion_jun_jul_solo_ipc_mayo.png"
plt.savefig(ruta_uf, bbox_inches='tight', dpi=150)
plt.close()

print(f"Gráfico de proyección UF para junio-julio (solo con IPC de mayo) guardado: {ruta_uf}")

# --- 4. Futuros ---
S0_futuros = 100.0
r_futuros = 0.04
T_futuros = 0.5
q_futuros = 0.015

def calculate_futures_price(S0, r, T, q=0):
    return S0 * np.exp((r - q) * T)

valor_futuros = calculate_futures_price(S0_futuros, r_futuros, T_futuros, q_futuros)

resumen_futuros = {
    "descripcion": "Este es el precio al que un contrato de futuros debería negociarse hoy, dado el precio spot actual del activo subyacente, la tasa libre de riesgo, el tiempo al vencimiento y los costos o rendimientos de carry.",
    "detalles": [
        f"Precio Spot (S0): ${S0_futuros:,.2f}",
        f"Tasa Libre de Riesgo (r): {r_futuros*100:.2f}%",
        f"Tiempo al Vencimiento (T): {T_futuros:.2f} años ({T_futuros*12:.0f} meses)",
        f"Tasa de Dividendo Continua (q): {q_futuros*100:.2f}%"
    ],
    "resultado": f"El Precio Teórico del Contrato de Futuros (F0) es: ${valor_futuros:,.2f}",
    "analisis": "Una comparación con el precio de mercado real revelaría si el contrato está sobrevalorado, subvalorado o justamente valorado."
}

s0_range_futuros_plot = np.linspace(S0_futuros * 0.8, S0_futuros * 1.2, 100)
futures_prices_s0_plot = [calculate_futures_price(s, r_futuros, T_futuros, q_futuros) for s in s0_range_futuros_plot]
plt.figure(figsize=(10, 6))
plt.plot(s0_range_futuros_plot, futures_prices_s0_plot, label='Precio de Futuros', color='darkblue')
plt.axvline(S0_futuros, color='red', linestyle='--', label=f'S0 actual (${S0_futuros:,.2f})')
plt.axhline(valor_futuros, color='green', linestyle=':', label=f'F0 teórico (${valor_futuros:,.2f})')
plt.title('Futuros: Precio de Futuros vs. Precio Spot del Activo Subyacente')
plt.xlabel('Precio Spot (S0) ($)')
plt.ylabel('Precio de Futuros (F0) ($)')
plt.legend()
plt.scatter(S0_futuros, valor_futuros, color='red', s=100, zorder=5)
plt.annotate(f'(${S0_futuros:,.2f}, ${valor_futuros:,.2f})', (S0_futuros, valor_futuros), textcoords="offset points", xytext=(0,10), ha='center', color='red')
ruta_futuros = "futuros_precio_vs_spot.png"
plt.savefig(ruta_futuros, bbox_inches='tight', dpi=150)
plt.close()

# --- 5. Forwards ---
def calculate_forward_price(S0, tasa_base, tasa_cotizada, n_dias, dias_base_base=30, dias_base_cotizada=360):
    return S0 * (1 + tasa_base * (n_dias / dias_base_base)) / (1 + tasa_cotizada * (n_dias / dias_base_cotizada))

# Escenario 1: Importador
S0_forward_importador = 740.0
tasa_clp_forward_importador = 0.009
tasa_usd_forward_importador = 0.023
n_dias_forward_importador = 90
valor_forward_importador = calculate_forward_price(S0_forward_importador, tasa_clp_forward_importador, tasa_usd_forward_importador, n_dias_forward_importador, 30, 360)

# Escenario 2: Codelco
S0_forward_codelco = 750.0
tasa_clp_forward_codelco = 0.008
tasa_usd_forward_codelco = 0.024
n_dias_forward_codelco = 90
valor_forward_codelco = calculate_forward_price(S0_forward_codelco, tasa_clp_forward_codelco, tasa_usd_forward_codelco, n_dias_forward_codelco, 30, 360)

resumen_forward = {
    "descripcion": "Un contrato Forward es un acuerdo personalizado para comprar o vender un activo a un precio específico en una fecha futura. A diferencia de los futuros, no se negocian en bolsa y conllevan riesgo de contraparte.",
    "subsecciones": [
        {
            "titulo": "Escenario 1: Forward de Venta (Importador)",
            "descripcion": "El importador necesita comprar USD a futuro para pagar mercadería. Al entrar en un forward de venta, asegura el precio al que comprará esos dólares.",
            "detalles": [
                f"Precio Spot Inicial (CLP/USD): ${S0_forward_importador:,.2f}",
                f"Tasa Interés CLP (ej. Colocación): {tasa_clp_forward_importador*100:.2f}% (base 30 días)",
                f"Tasa Interés USD (ej. Captación): {tasa_usd_forward_importador*100:.2f}% (base 360 días)",
                f"Plazo: {n_dias_forward_importador} días"
            ],
            "resultado": f"El Precio de Forward de Venta para el Importador es: ${valor_forward_importador:,.2f} CLP/USD."
        },
        {
            "titulo": "Escenario 2: Forward de Compra (Codelco)",
            "descripcion": "Codelco, como exportador, recibirá USD a futuro y desea asegurar el precio al que los convertirá a CLP. Al entrar en un forward de compra, fija el tipo de cambio de venta.",
            "detalles": [
                f"Precio Spot Inicial (CLP/USD): ${S0_forward_codelco:,.2f}",
                f"Tasa Interés CLP (ej. Captación): {tasa_clp_forward_codelco*100:.2f}% (base 30 días)",
                f"Tasa Interés USD (ej. Colocación): {tasa_usd_forward_codelco*100:.2f}% (base 360 días)",
                f"Plazo: {n_dias_forward_codelco} días"
            ],
            "resultado": f"El Precio de Forward de Compra para Codelco es: ${valor_forward_codelco:,.2f} CLP/USD."
        }
    ],
    "analisis": "La diferencia entre el spot y el forward (puntos forward) está determinada principalmente por el diferencial de tasas de interés entre las dos divisas y el plazo de la operación. Un forward permite a las empresas y a los individuos mitigar el riesgo de tipo de cambio futuro."
}

s0_range_forward_plot = np.linspace(S0_forward_importador * 0.9, S0_forward_importador * 1.1, 100)
forward_prices_s0_plot = [calculate_forward_price(s, tasa_clp_forward_importador, tasa_usd_forward_importador, n_dias_forward_importador, 30, 360) for s in s0_range_forward_plot]
plt.figure(figsize=(10, 6))
plt.plot(s0_range_forward_plot, forward_prices_s0_plot, label='Precio Forward (Importador)', color='darkorange')
plt.axvline(S0_forward_importador, color='red', linestyle='--', label=f'S0 actual (${S0_forward_importador:,.2f})')
plt.axhline(valor_forward_importador, color='green', linestyle=':', label=f'Forward teórico (${valor_forward_importador:,.2f})')
plt.title('Forward: Precio Forward vs. Precio Spot Inicial (Importador)')
plt.xlabel('Precio Spot Inicial (CLP/USD)')
plt.ylabel('Precio de Forward (CLP/USD)')
plt.legend()
plt.scatter(S0_forward_importador, valor_forward_importador, color='red', s=100, zorder=5)
plt.annotate(f'(${S0_forward_importador:,.2f}, ${valor_forward_importador:,.2f})', (S0_forward_importador, valor_forward_importador), textcoords="offset points", xytext=(0,10), ha='center', color='red')
ruta_forward = "forward_precio_vs_spot.png"
plt.savefig(ruta_forward, bbox_inches='tight', dpi=150)
plt.close()

# --- 6. Opciones ---
S0_opcion = 10.0
u_opcion = 1.2
d_opcion = 0.8
K_opcion = 8.0
r_opcion = 0.11 # Tasa libre de riesgo por período (importante: debe ser por período si u, d son por período)
p_opcion = 0.75 # Probabilidad de movimiento al alza
tipo_opcion_opcion = "Call" # O "Put"

# --- Función para calcular el Modelo Binomial de 2 Pasos ---
def calculate_option_binomial_2step(S0, K, u, d, r, p, tipo_opcion="Call"):
    """
    Calcula los precios del activo subyacente y los valores de la opción
    en un modelo binomial de 2 pasos.
    """
    # Precios del activo subyacente en cada nodo
    Su, Sd = S0 * u, S0 * d
    Suu, Sud, Sdd = Su * u, Su * d, Sd * d

    # Valores de la opción en el vencimiento (t=2)
    if tipo_opcion.lower() == "call":
        V_uu = max(0, Suu - K)
        V_ud = max(0, Sud - K)
        V_dd = max(0, Sdd - K)
    else: # Put
        V_uu = max(0, K - Suu)
        V_ud = max(0, K - Sud)
        V_dd = max(0, K - Sdd)

    # Valores de la opción en t=1 (descontando de t=2)
    V_u = (p * V_uu + (1 - p) * V_ud) / (1 + r)
    V_d = (p * V_ud + (1 - p) * V_dd) / (1 + r)

    # Valor de la opción en t=0 (descontando de t=1)
    V0 = (p * V_u + (1 - p) * V_d) / (1 + r)

    return V0, V_u, V_d, V_uu, V_ud, V_dd, S0, Su, Sd, Suu, Sud, Sdd

# --- Ejecutar el cálculo del modelo binomial ---
(valor_opcion, Cu, Cd, Cuu, Cud, Cdd, S0_val, Su_val, Sd_val, Suu_val, Sud_val, Sdd_val) = \
    calculate_option_binomial_2step(S0_opcion, K_opcion, u_opcion, d_opcion, r_opcion, p_opcion, tipo_opcion_opcion)

# --- Resumen para el PDF (mantener el formato original) ---
resumen_opcion = {
    "descripcion": "Una opción es un contrato que otorga a su comprador el derecho, pero no la obligación, de comprar o vender un activo subyacente a un precio determinado (precio de ejercicio) en una fecha futura específica (fecha de vencimiento).\n\nEste análisis utiliza un Modelo Binomial de 2 pasos para valorar una Call Europea, mostrando tanto el árbol de precios del activo subyacente como el árbol de valoración de la opción.",
    "detalles": [
        f"Tipo de Opción: {tipo_opcion_opcion} Europea",
        f"Precio Spot Inicial (S0): ${S0_opcion:,.2f}",
        f"Precio de Ejercicio (K): ${K_opcion:,.2f}",
        f"Factor de Movimiento al Alza (u): {u_opcion:.2f}",
        f"Factor de Movimiento a la Baja (d): {d_opcion:.2f}",
        f"Tasa Libre de Riesgo por Período (r): {r_opcion*100:.2f}%",
        f"Probabilidad de Alza (p): {p_opcion:.2f}"
    ],
    "resultado": f"Valor Intrínseco Calculado de la Opción en t=0: ${valor_opcion:,.2f}",
    "analisis": "El valor de la opción se deriva del árbol binomial que proyecta los posibles precios futuros del activo subyacente y, en retrospectiva, el valor de la opción en cada nodo. Un modelo binomial es flexible y puede adaptarse a opciones americanas, dividendos y otras características complejas. En este caso, el valor de ${valor_opcion:,.2f} representa el precio justo de la opción hoy, basado en los parámetros dados."
}

# --- Función para generar los gráficos de árbol binomial (sin cambios) ---
def generar_graficos_arbol_binomial(S0, Su, Sd, Suu, Sud, Sdd, C0, Cu, Cd, Cuu, Cud, Cdd, ruta_precio, ruta_valor):
    pos = {
        "t0": (0, 0),
        "t1u": (1, 1), "t1d": (1, -1),
        "t2uu": (2, 1.5), "t2ud": (2, 0), "t2dd": (2, -1.5),
    }

    # Árbol de Precios del Activo Subyacente
    G_price = nx.DiGraph()
    edges_price = [("t0", "t1u"), ("t0", "t1d"),
                   ("t1u", "t2uu"), ("t1u", "t2ud"),
                   ("t1d", "t2ud"), ("t1d", "t2dd")]
    G_price.add_edges_from(edges_price)

    labels_price = {
        "t0": f"S0\n${S0:,.2f}",
        "t1u": f"Su\n${Su:,.2f}",
        "t1d": f"Sd\n${Sd:,.2f}",
        "t2uu": f"Suu\n${Suu:,.2f}",
        "t2ud": f"Sud\n${Sud:,.2f}",
        "t2dd": f"Sdd\n${Sdd:,.2f}"
    }

    plt.figure(figsize=(12, 8))
    nx.draw(G_price, pos, with_labels=False,
            node_color="lightblue", node_size=6000,
            edge_color='gray', width=1.5,
            arrows=True, arrowstyle='-|>', arrowsize=30,
            font_size=12, font_weight='bold')
    nx.draw_networkx_labels(G_price, pos, labels_price, font_size=12, font_weight='bold')

    plt.title("Árbol Binomial del Precio del Activo Subyacente", fontsize=18)
    plt.axis("off")
    plt.savefig(ruta_precio, bbox_inches='tight', dpi=200)
    plt.close()

    # Árbol del Valor de la Opción
    G_valor = nx.DiGraph()
    G_valor.add_edges_from(edges_price)

    labels_valor = {
        "t0": f"C0\n${C0:,.2f}",
        "t1u": f"Cu\n${Cu:,.2f}",
        "t1d": f"Cd\n${Cd:,.2f}",
        "t2uu": f"Cuu\n${Cuu:,.2f}",
        "t2ud": f"Cud\n${Cud:,.2f}",
        "t2dd": f"Cdd\n${Cdd:,.2f}"
    }

    plt.figure(figsize=(12, 8))
    nx.draw(G_valor, pos, with_labels=False,
            node_color="lightgreen", node_size=6000,
            edge_color='gray', width=1.5,
            arrows=True, arrowstyle='-|>', arrowsize=30,
            font_size=12, font_weight='bold')
    nx.draw_networkx_labels(G_valor, pos, labels_valor, font_size=12, font_weight='bold', font_color='darkgreen')

    plt.title(f"Árbol Binomial del Valor de la Opción {tipo_opcion_opcion} Europea", fontsize=18)
    plt.axis("off")
    plt.savefig(ruta_valor, bbox_inches='tight', dpi=200)
    plt.close()

# --- Rutas de archivo para los gráficos ---
ruta_opciones_precio = "opciones_arbol_precio.png"
ruta_opciones_valor = "opciones_arbol_valor.png"

# --- Generar los gráficos de árbol binomial ---
generar_graficos_arbol_binomial(
    S0_val, Su_val, Sd_val, Suu_val, Sud_val, Sdd_val,
    valor_opcion, Cu, Cd, Cuu, Cud, Cdd,
    ruta_opciones_precio, ruta_opciones_valor
)

# --- NUEVO Gráfico de Payoff para Call (similar al estilo solicitado) ---
# Rango de precios del activo subyacente para el gráfico de payoff
# Se extiende más allá del strike para mostrar la ganancia ilimitada
s_range_payoff = np.linspace(K_opcion * 0.5, K_opcion * 2.0, 200) # Rango más amplio centrado en K

# Calcular el payoff de una Call larga
# Payoff = max(0, S_T - K) - Prima pagada
# Aquí, usaremos valor_opcion como la prima, asumiendo que es el costo de adquirir la opción.
if tipo_opcion_opcion.lower() == "call":
    payoff_values = np.maximum(0, s_range_payoff - K_opcion) - valor_opcion
else: # Tipo Put
    payoff_values = np.maximum(0, K_opcion - s_range_payoff) - valor_opcion


plt.figure(figsize=(10, 6))

# Línea del payoff (verde para ganancia, rojo para pérdida)
# Seccionar la línea donde el payoff es negativo y donde es positivo
plt.plot(s_range_payoff, payoff_values, color='green', linewidth=2, label=f'Ganancia Neta {tipo_opcion_opcion}')
plt.plot(s_range_payoff[payoff_values < 0], payoff_values[payoff_values < 0], color='red', linewidth=2, label=f'Pérdida Neta {tipo_opcion_opcion}')


# Línea horizontal para la prima pagada (en negativo, ya que es un costo)
plt.axhline(-valor_opcion, color='orange', linestyle='--', linewidth=1.5, label=f'Prima Pagada (${valor_opcion:,.2f})')

# Línea vertical para el Precio de Ejercicio (K)
plt.axvline(K_opcion, color='gray', linestyle='-.', linewidth=1.5,
            label=f'Precio de Ejercicio (PE) (${K_opcion:,.2f})')

# Línea horizontal en cero para la ganancia/pérdida neta
plt.axhline(0, color='black', linestyle='-', linewidth=1)

# Títulos y etiquetas
plt.title(f'Payoff de la Compra de una Opción {tipo_opcion_opcion} Europea', fontsize=16)
plt.xlabel('Precio del Activo Subyacente al Vencimiento ($)', fontsize=12)
plt.ylabel('Ganancia / Pérdida Neta ($)', fontsize=12)

# Añadir punto de equilibrio (break-even point)
if tipo_opcion_opcion.lower() == "call":
    break_even_point = K_opcion + valor_opcion
else: # Put
    break_even_point = K_opcion - valor_opcion

plt.plot(break_even_point, 0, 'o', color='purple', markersize=8, label=f'Punto de Equilibrio (${break_even_point:,.2f})')
plt.annotate(f'PE: ${break_even_point:,.2f}',
             (break_even_point, 0),
             textcoords="offset points", xytext=(5,-15), ha='center',
             color='purple', fontsize=10)

# Leyenda
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)

# Ajustar límites del eje X e Y para un mejor visual
plt.xlim(s_range_payoff.min(), s_range_payoff.max())
# El límite Y debe incluir la prima negativa y el máximo payoff esperado
plt.ylim(min(-valor_opcion * 1.5, payoff_values.min()), max(payoff_values.max() * 1.1, valor_opcion * 0.5))

ruta_opciones_sensibilidad = "opciones_payoff_call_sensibilidad.png"
plt.savefig(ruta_opciones_sensibilidad, bbox_inches='tight', dpi=200)
plt.close()

print(f"Gráficos de árbol binomial guardados: {ruta_opciones_precio} y {ruta_opciones_valor}")
print(f"Gráfico de Payoff de opciones mejorado guardado: {ruta_opciones_sensibilidad}")

# --- 7. Swaps ---
notional_swap = 1_000_000
fixed_rate_swap = 0.04
floating_rate_current_swap = 0.045
N_years_swap = 5
payments_per_year_swap = 2
discount_rate_swap = 0.038

def calculate_swap_value(notional, fixed_rate, floating_rate, N_years, payments_per_year, discount_rate):
    period_length = 1 / payments_per_year
    total_payments = N_years * payments_per_year
    fixed_leg_pv = sum([(notional * fixed_rate * period_length) / (1 + discount_rate)**((i+1)*period_length) for i in range(total_payments)])
    floating_leg_pv = sum([(notional * floating_rate * period_length) / (1 + discount_rate)**((i+1)*period_length) for i in range(total_payments)])
    return fixed_leg_pv - floating_leg_pv

valor_swap = calculate_swap_value(notional_swap, fixed_rate_swap, floating_rate_current_swap, N_years_swap, payments_per_year_swap, discount_rate_swap)

resumen_swap = {
    "descripcion": "Un Interest Rate Swap (IRS) es un acuerdo entre dos partes para intercambiar flujos de intereses futuros, uno basado en una tasa fija y otro en una tasa flotante, sobre un valor nominal nocional. Permite a las empresas gestionar su exposición a las tasas de interés.",
    "detalles": [
        f"Valor Nominal Nocional: ${notional_swap:,.2f}",
        f"Tasa Fija Anual (Recibida): {fixed_rate_swap*100:.2f}%",
        f"Tasa Flotante Actual (Pagada): {floating_rate_current_swap*100:.2f}%",
        f"Duración del Swap: {N_years_swap} años",
        f"Frecuencia de Pagos: {payments_per_year_swap} veces al año ({12/payments_per_year_swap:.0f} meses)",
        f"Tasa de Descuento Promedio (Curva plana): {discount_rate_swap*100:.2f}%"
    ],
    "resultado": f"Valor Neto Presente del Swap: ${valor_swap:,.2f}",
    "analisis": f"Este valor representa el beneficio o costo neto de entrar en el swap para la parte que recibe la tasa fija y paga la tasa flotante.\n- Si el valor es positivo, el swap tiene valor para el receptor de la pata fija (beneficio).\n- Si el valor es negativo, el swap tiene un costo para el receptor de la pata fija (pérdida).\nEn este caso, con una tasa flotante actual del {floating_rate_current_swap*100:.2f}% y una tasa fija del {fixed_rate_swap*100:.2f}%, el swap tiene un valor de ${valor_swap:,.2f}. Si la tasa flotante sube por encima de la tasa fija, el valor para el receptor de la fija se vuelve más positivo (o menos negativo), y viceversa."
}

libor_range_swap_plot = np.linspace(0.02, 0.07, 100)
valores_neto_swap_plot = [calculate_swap_value(notional_swap, fixed_rate_swap, libor_val, N_years_swap, payments_per_year_swap, discount_rate_swap) for libor_val in libor_range_swap_plot]
plt.figure(figsize=(10, 6))
plt.plot(libor_range_swap_plot * 100, valores_neto_swap_plot, color="blue", label="Valor Neto del Swap")
plt.title("Swaps: Valor del Swap vs. Tasa Flotante (LIBOR/SOFR)")
plt.xlabel("Tasa Flotante (LIBOR/SOFR) (%)")
plt.ylabel("Valor Neto del Swap ($)")
plt.axhline(0, color='black', linestyle='--', label='Valor Cero')
plt.legend()
plt.axvline(floating_rate_current_swap * 100, color='red', linestyle='--', label=f'Tasa Flotante Actual: {floating_rate_current_swap*100:.2f}%')
plt.axhline(valor_swap, color='green', linestyle=':', label=f'Valor Actual del Swap: ${valor_swap:,.2f}')
plt.scatter(floating_rate_current_swap * 100, valor_swap, color='red', s=100, zorder=5)
plt.annotate(f'({floating_rate_current_swap*100:.2f}%, ${valor_swap:,.2f})', (floating_rate_current_swap * 100, valor_swap), textcoords="offset points", xytext=(0,10), ha='center', color='red')
ruta_swaps = "swaps_sensibilidad_tasa_flotante.png"
plt.savefig(ruta_swaps, bbox_inches='tight', dpi=150)
plt.close()

# --- 8. FRAs ---
def calculate_fra_rate(rate_6_months, rate_9_months):
    """
    Calculates the Forward Rate Agreement (FRA) rate for a 6x9 FRA.

    Args:
        rate_6_months (float): The linear annual interest rate for 6 months (as a decimal).
        rate_9_months (float): The linear annual interest rate for 9 months (as a decimal).

    Returns:
        float: The calculated FRA rate (as a decimal).
    """

    fv_9_months = 1 + (rate_9_months * 9 / 12)
    fv_6_months = 1 + (rate_6_months * 6 / 12)

    fra_rate = ((fv_9_months / fv_6_months) - 1) * (12 / 3) # (12 / 3) because it's a 3-month FRA period

    return fra_rate

# Given values from the image
rate_6_months_percent = 2.0  # 2%
rate_9_months_percent = 2.5  # 2.5%

# Convert percentages to decimals
rate_6_months_decimal = rate_6_months_percent / 100
rate_9_months_decimal = rate_9_months_percent / 100

# Calculate the FRA rate
fra_result_decimal = calculate_fra_rate(rate_6_months_decimal, rate_9_months_decimal)
fra_result_percent = fra_result_decimal * 100

print(f"La tasa FRA 6x9 es: {fra_result_percent:.2f}%")

# --- Plotting the graph ---

# Data for plotting
labels = ['6-month', '9-month', '6x9 FRA']
rates = [rate_6_months_percent, rate_9_months_percent, fra_result_percent]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(8, 6))
rects1 = ax.bar(x, rates, width, label='Interest Rate', color=['#4F81BD', '#F79646', '#4F81BD'])

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Interest Rate (%)')
ax.set_title('Interés: Comparación de Tasas a Término') # Changed title for clarity
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylim(0, max(rates) * 1.2) # Set y-axis limit slightly above max rate

# Add value labels on top of the bars
for rect in rects1:
    height = rect.get_height()
    ax.annotate(f'{height:.2f}%',
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

# Hide the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.grid(axis='y', linestyle='--', alpha=0.7) # Add subtle grid lines on y-axis
plt.tight_layout()
ruta_fra = "fra_tasas_comparacion.png" # Define the image path for FRA
plt.savefig(ruta_fra, bbox_inches='tight', dpi=150)
plt.close()

# Define the resumen_fra dictionary
resumen_fra = {
    "descripcion": "Un Forward Rate Agreement (FRA) es un contrato de derivados over-the-counter (OTC) que permite a dos partes acordar una tasa de interés para un período futuro predeterminado sobre un monto nocional. Se utiliza para protegerse contra el riesgo de fluctuaciones en las tasas de interés.",
    "detalles": [
        f"Tasa Spot a 6 meses: {rate_6_months_percent:.2f}%",
        f"Tasa Spot a 9 meses: {rate_9_months_percent:.2f}%",
        f"Periodo FRA: 3 meses (entre el mes 6 y el mes 9)"
    ],
    "resultado": f"La Tasa de Forward Rate Agreement (FRA) 6x9 es: {fra_result_percent:.2f}%",
    "analisis": f"La tasa FRA 6x9 de {fra_result_percent:.2f}% representa la tasa de interés implícita a 3 meses que se espera prevalezca en el mercado a partir de dentro de 6 meses. Este cálculo es crucial para empresas y bancos que desean fijar las tasas de interés de préstamos o inversiones futuras, mitigando la incertidumbre de las tasas del mercado."
}

# =========================================================================
# === PASO FINAL: Generación del Informe PDF ==============================
# =========================================================================

class PDFReport(FPDF):
    def header(self):
        self.set_y(10)
        logo_path = "UsachP2.png"  # Usar el logo descargado
        if os.path.exists(logo_path):
            self.image(logo_path, x=self.w - 35, y=8, w=25)
        else:
            self.set_font('Arial', 'B', 8)
            self.set_text_color(200, 0, 0)
            self.cell(0, 5, f"ADVERTENCIA: Logo '{logo_path}' no encontrado.", 0, 1, 'R')
            self.set_text_color(0, 0, 0)
        if self.page_no() > 1:
            self.ln(20)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}/{{nb}}', 0, 0, 'C')

    def crear_tabla(self, data, col_widths):
        self.set_font('Arial', 'B', 9)
        self.set_fill_color(224, 235, 255)
        for i, header in enumerate(data[0]):
            self.cell(col_widths[i], 7, header, 1, 0, 'C', 1)
        self.ln()
        self.set_font('Arial', '', 9)
        for row in data[1:]:
            for i, item in enumerate(row):
                align = 'L' if i == 0 else 'R'
                self.cell(col_widths[i], 6, str(item), 1, 0, align)
            self.ln()
        self.ln(5)

    def crear_indice(self, secciones):
        self.add_page()
        self.set_font('Arial', 'B', 16)
        self.cell(0, 15, 'Índice', 0, 1, 'C')
        self.ln(10)

        self.set_font('Arial', '', 12)
        pagina_actual = 2

        titulo_intro = "Contenidos del informe"
        self.cell(0, 10, f'{titulo_intro}', 0, 0, 'L')
        self.cell(0, 10, str(pagina_actual), 0, 1, 'R')
        pagina_actual += 1

        for titulo_seccion in secciones:
            ancho_titulo = self.get_string_width(titulo_seccion) + 2
            ancho_puntos = self.w - self.l_margin - self.r_margin - ancho_titulo - 10
            puntos = '.' * int(ancho_puntos / self.get_string_width('.'))

            self.cell(ancho_titulo, 10, titulo_seccion, 0, 0, 'L')
            self.cell(ancho_puntos, 10, puntos, 0, 0, 'L')
            self.cell(10, 10, str(pagina_actual), 0, 1, 'R')
            pagina_actual += 1

    def agregar_seccion(self, titulo, data, imagenes_paths):
        self.add_page()
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, titulo, 0, 1, 'L')
        self.ln(2)

        if data.get("descripcion"):
            self.set_font('Arial', '', 10)
            self.multi_cell(0, 5, data["descripcion"].encode('latin-1', 'replace').decode('latin-1'))
            self.ln(4)

        if data.get("subsecciones"):
            for sub in data["subsecciones"]:
                self.set_font('Arial', 'B', 11)
                self.cell(0, 6, sub["titulo"], 0, 1)
                self.set_font('Arial', '', 10)
                self.multi_cell(0, 5, sub["descripcion"].encode('latin-1', 'replace').decode('latin-1'))
                self.ln(2)
                for item in sub["detalles"]:
                    self.multi_cell(0, 5, f"    •   {item}".encode('latin-1', 'replace').decode('latin-1'))
                self.set_font('Arial', 'B', 10)
                self.multi_cell(0, 5, sub["resultado"].encode('latin-1', 'replace').decode('latin-1'))
                self.ln(4)
            self.ln(2)

        if data.get("detalles") and not data.get("subsecciones"):
            self.set_font('Arial', 'B', 10)
            self.cell(0, 6, "Detalles del Activo:", 0, 1)
            self.set_font('Arial', '', 10)
            for item in data["detalles"]:
                self.multi_cell(0, 5, f"    •   {item}".encode('latin-1', 'replace').decode('latin-1'))
            self.ln(4)

        if data.get("tabla"):
            self.set_font('Arial', 'B', 10)
            self.cell(0, 6, "Proyección de Valores:", 0, 1)
            self.crear_tabla(data["tabla"], col_widths=[70, 100])

        if data.get("resultado"):
            self.set_font('Arial', 'B', 10)
            self.multi_cell(0, 6, data["resultado"].encode('latin-1', 'replace').decode('latin-1'))
            self.ln(4)

        if data.get("analisis"):
            self.set_font('Arial', 'B', 10)
            self.cell(0, 6, "Análisis:", 0, 1)
            self.set_font('Arial', '', 10)
            self.multi_cell(0, 5, data["analisis"].encode('latin-1', 'replace').decode('latin-1'))
            self.ln(5)

        for img_path in imagenes_paths:
            if os.path.exists(img_path):
                if self.get_y() > 200:
                    self.add_page()
                self.image(img_path, w=170)
                self.ln(5)
            else:
                self.set_text_color(200, 0, 0)
                self.multi_cell(0, 5, f"Error: No se encontró la imagen '{img_path}'.")
                self.set_text_color(0, 0, 0)

# --- Creación del PDF ---
# --- Creación del PDF ---
pdf = PDFReport()
pdf.alias_nb_pages()
pdf.set_auto_page_break(auto=True, margin=25)

# Página de Título con nombre del profesor agregado
pdf.add_page()
pdf.set_y(80)
pdf.set_font('Arial', 'B', 28)
pdf.multi_cell(0, 12, 'Informe Valoración\nde Activos Financieros', 0, 'C')
pdf.ln(15)
pdf.set_font('Arial', '', 16)
pdf.multi_cell(0, 10, 'Profesor: Carlos Cavieres', 0, 'C')  # Nombre del profesor agregado
pdf.ln(10)
pdf.set_font('Arial', '', 16)
pdf.multi_cell(0, 10, 'Rodrigo Alvear - Camila Bravo\nJeremy Bustamante - Anthony Iturrieta', 0, 'C')
pdf.ln(5)
pdf.set_font('Arial', 'I', 12)
fecha_formateada = obtener_fecha_en_espanol()
pdf.cell(0, 10, f"Fecha de Emisión: {fecha_formateada}", 0, 1, 'C')

# Crear el índice
lista_secciones_informe = [
    "Introducción", "1. Bonos", "2. Acciones", "3. Unidad de Fomento (UF)", "4. Futuros",
    "5. Forwards", "6. Opciones", "7. Swaps", "8. FRA (Forward Rate Agreement)", "Conclusión"
]
pdf.crear_indice(lista_secciones_informe)

# Introducción
pdf.add_page()
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, "Introducción", 0, 1, 'L')
pdf.set_font('Arial', '', 10)
intro_text = """El presente informe ofrece un análisis exhaustivo y una valoración detallada de un portafolio diversificado de activos financieros. En un entorno económico global caracterizado por la volatilidad de los mercados, las fluctuaciones en las tasas de interés y la incertidumbre geopolítica, la capacidad de determinar el valor intrínseco de los instrumentos financieros se convierte en una herramienta fundamental para la toma de decisiones estratégicas, la gestión de riesgos y la identificación de oportunidades de inversión.

El objetivo principal de este documento es aplicar modelos de valoración financiera estándar de la industria para estimar el precio teórico de ocho clases de activos distintas: desde instrumentos de renta fija como los bonos, hasta renta variable representada por acciones, pasando por derivados complejos como futuros, forwards, opciones, swaps y FRAs. Adicionalmente, se incluye un análisis de la Unidad de Fomento (UF) de Chile, un instrumento indexado a la inflación de crucial importancia para la economía local.

Para cada activo, se presenta una descripción conceptual, los parámetros y supuestos utilizados en el cálculo, el resultado de la valoración y un análisis interpretativo. Además, se incluyen gráficos de sensibilidad que ilustran cómo el valor del activo responde a cambios en variables clave del mercado, como las tasas de interés, las tasas de crecimiento o los precios del subyacente. Esta aproximación no solo provee un valor numérico, sino que también fomenta una comprensión más profunda de la dinámica de precios de cada instrumento.

Este informe está dirigido a estudiantes, analistas financieros y profesionales del sector que busquen consolidar su comprensión teórica y práctica de las técnicas de valoración de activos. Los cálculos y visualizaciones han sido generados mediante un script de Python, garantizando la reproducibilidad y transparencia de los resultados."""
pdf.multi_cell(0, 5, intro_text.encode('latin-1', 'replace').decode('latin-1'))

# Agregar cada activo al informe
pdf.agregar_seccion("1. Bonos", resumen_bono, [ruta_bonos])
pdf.agregar_seccion("2. Acciones", resumen_accion, [ruta_acciones])
pdf.agregar_seccion("3. Unidad de Fomento (UF)", resumen_uf, [ruta_uf])
pdf.agregar_seccion("4. Futuros", resumen_futuros, [ruta_futuros])
pdf.agregar_seccion("5. Forwards", resumen_forward, [ruta_forward])
pdf.agregar_seccion("6. Opciones", resumen_opcion, [ruta_opciones_precio, ruta_opciones_valor, ruta_opciones_sensibilidad])
pdf.agregar_seccion("7. Swaps", resumen_swap, [ruta_swaps])
pdf.agregar_seccion("8. FRA (Forward Rate Agreement)", resumen_fra, [ruta_fra]) # Now resumen_fra and ruta_fra are defined

# Conclusión
pdf.add_page()
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, "Conclusión", 0, 1, 'L')
pdf.set_font('Arial', '', 10)
intro_text = """A través de la valoración de ocho clases de activos distintas, este informe ha demostrado la aplicación práctica de modelos financieros fundamentales. Se ha evidenciado que el valor de un instrumento no es estático, sino que responde dinámicamente a cambios en variables clave como las tasas de interés, las expectativas de crecimiento, la inflación y la volatilidad del mercado.

Los gráficos de sensibilidad, en particular, han subrayado que la relación entre estas variables y el precio de un activo rara vez es lineal. Instrumentos como las opciones exhiben una convexidad que es crucial para su gestión de riesgo y estrategia. Se ha observado que, si bien los modelos como Black-Scholes o el de Crecimiento de Gordon proporcionan una base teórica sólida, su precisión depende críticamente de la calidad de los supuestos, especialmente de la estimación de la volatilidad y las tasas de crecimiento.

En última instancia, este análisis refuerza la idea de que la valoración de activos no es una ciencia exacta, sino una disciplina que combina rigor matemático con juicio de mercado. Los precios teóricos calculados son una guía, no un veredicto. La verdadera habilidad de un analista financiero reside en comprender las limitaciones de estos modelos y en interpretar las desviaciones entre el valor teórico y el precio de mercado para identificar riesgos y oportunidades. Este informe sirve, por tanto, como una base sólida para desarrollar esa habilidad analítica indispensable en el mundo de las finanzas."""
pdf.multi_cell(0, 5, intro_text.encode('latin-1', 'replace').decode('latin-1'))

# Guardar PDF
pdf_output_filename = "Informe_Valoracion_Financiera_Completo.pdf"
try:
    pdf.output(pdf_output_filename)
    print(f"✅ ¡Éxito! Informe completo generado: {pdf_output_filename}")

    # Limpiar archivos de imagen generados
    rutas_imagenes = [
        ruta_bonos, ruta_acciones, ruta_uf, ruta_futuros, ruta_forward,
        ruta_opciones_precio, ruta_opciones_valor, ruta_opciones_sensibilidad,
        ruta_swaps, ruta_fra
    ]
    for ruta in rutas_imagenes:
        if os.path.exists(ruta):
            os.remove(ruta)
    print("🧹 Archivos de imagen temporales eliminados.")

except Exception as e:
    print(f"❌ Error al generar el PDF: {e}")
    print("Asegúrese de no tener el archivo PDF abierto mientras ejecuta el script.")

# Descargar en Colab
if 'google.colab' in sys.modules:
    from google.colab import files
    files.download(pdf_output_filename)

Gráfico de proyección UF para junio-julio (solo con IPC de mayo) guardado: uf_proyeccion_jun_jul_solo_ipc_mayo.png
Gráficos de árbol binomial guardados: opciones_arbol_precio.png y opciones_arbol_valor.png
Gráfico de Payoff de opciones mejorado guardado: opciones_payoff_call_sensibilidad.png
La tasa FRA 6x9 es: 3.47%
✅ ¡Éxito! Informe completo generado: Informe_Valoracion_Financiera_Completo.pdf
🧹 Archivos de imagen temporales eliminados.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>